In [1]:
import networkx as nx
from functools import cache

In [2]:
with open('../data/2024/day21.txt') as f:
    data = f.read()

In [3]:
codes = data.splitlines()

In [4]:
# Number pad adjacency (favor: left, down, right, up)
numpad_adj = """
A0<,A3^,0A>,02^,12>,14^,21<,20v,23>,25^,32<,3Av,36^,41v,45>,47^,
54<,52v,56>,58^,65<,63v,69^,74v,78>,87<,85v,89>,98<,96v
""".replace('\n','').split(',')
G_numpad = nx.DiGraph()
for u,v,k in list(numpad_adj): G_numpad.add_edge(u,v,key=k)

In [5]:
@cache
def shortest_path_numpad(u, v, depth):
    paths = list(nx.all_shortest_paths(G_numpad, u, v))
    results = []

    for path in paths:
        dirs = []
        for a, b in zip(path, path[1:]):
            dirs.append(G_numpad[a][b]['key'])

        if dirs == ['>', '^', '>']: pass
        elif dirs == ['<', 'v', '<']: pass
        elif dirs == ['^', '<', '^', '^']: pass
        elif dirs == ['^', '^', '<', '^']: pass
        elif dirs == ['^', '^', '>', '^']: pass
        elif dirs == ['^', '>', '^', '^']: pass
        elif dirs == ['<', '^', '<', '^','^']: pass
        elif dirs == ['<', '^', '^', '<','^']: pass
        elif dirs == ['^', '<', '<', '^','^']: pass
        else:
            results.append(dirs)

    shortest_path_len = float('inf')

    for result in results:
        if 0 == depth:
            if len(result) < shortest_path_len:
                shortest_path_len = len(result)
        else:
            result = ['A'] + result + ['A']
            next_path_len = 0
            for next_u, next_v in zip(result, result[1:]):
                next_path_len += shortest_path_dirpad(next_u, next_v, depth)
            if next_path_len < shortest_path_len:
                shortest_path_len = next_path_len

    return shortest_path_len

In [6]:
# Directional pad
dirpad_adj = "A^<,A>v,^A>,^vv,v>>,v<<,v^^,<v>,>A^,>v<".split(',')
G_dirpad = nx.DiGraph()
for u,v,k in list(dirpad_adj): G_dirpad.add_edge(u,v,key=k)

In [7]:
@cache
def shortest_path_dirpad(u, v, depth):
    paths = list(nx.all_shortest_paths(G_dirpad, u, v))
    results = []

    for path in paths:
        dirs = []
        for a, b in zip(path, path[1:]):
            dirs.append(G_dirpad[a][b]['key'])

        if dirs == ['>', '^', '>']: pass
        elif dirs == ['<', 'v', '<']: pass
        else:
            dirs.append('A')
            results.append(dirs)

    shortest_path_len = float('inf')

    for result in results:
        if 0 == depth:
            if len(result) < shortest_path_len:
                shortest_path_len = len(result)
        else:
            result = ['A'] + result
            next_path_len = 0
            for next_u, next_v in zip(result, result[1:]):
                next_path_len += shortest_path_dirpad(next_u, next_v, depth-1)
            if next_path_len < shortest_path_len:
                shortest_path_len = next_path_len

    return shortest_path_len

In [8]:
def type_code(depth: int):
    count = 0

    for code in codes:
        start_code = list('A' + code)
        code_len = 0

        for u, v in zip(start_code, start_code[1:]):
            code_len += shortest_path_numpad(u, v, depth-1)
        count += code_len * int(''.join([d for d in code if d.isdigit()]))

    return count

In [9]:
print("Part 1:", type_code(2))
print("Part 2:", type_code(25))

Part 1: 164960
Part 2: 205620604017764
